In [221]:
from geopy.geocoders import Nominatim
import json
import requests as rq
from bs4 import BeautifulSoup

geolocator = Nominatim(user_agent="geoapiExercises")
cloud_t = ["--", "sun-bw", "sunc-bw", "suncl-bw", "dull-bw"]
phen_t = ["--", "rain-bw", "snow-bw", "storm-bw"]
wind_d = ["--", "С", "З", "Ю", "В", "СЗ", "СВ", "ЮЗ", "ЮВ"]
lat = "136.80700409"
lon = "49.93432764"
year = 2015
month = 10
day = 5

In [222]:
f = open('parsed_districts.json', "r")
county_codes = json.load(f)

In [223]:
location = geolocator.reverse(lon+","+lat)
county = location.raw["address"]["county"].replace(" район", "").replace("ое", "").replace("ий", "")
print("    "+county)
county_code = -1
try:
    county_code = county_codes[location.raw["address"]["state"]][county]
    print(county_code)
    print("found")
except:
    print("Code not found!")
    print(county_codes[location.raw["address"]["state"]])

    Амурск
11448
found


In [224]:
headers_={
        'accept': 'application/xml, text/xml, */*',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
        'content-type': 'application/x-www-form-urlencoded',
        'referer': 'https://www.gismeteo.ru/diary/206418/2015/10/',
        'sec-ch-ua': '"Chromium";v="92", " Not A;Brand";v="99", "Google Chrome";v="92"',
        'sec-ch-ua-mobile': '?0',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-origin',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36',
        'x-requested-with': 'XMLHttpRequest'}

response = rq.get(
    "https://www.gismeteo.ru/diary/"+str(county_code)+"/"+str(year)+"/"+str(month)+"/",
    headers=headers_)

In [225]:
soup = BeautifulSoup(response.text, 'html.parser')

In [226]:
rows = soup.find("tbody").find_all("tr")
result = {}
for row in rows:
    if row.find("td").text == str(day):
        colls = row.find_all("td")
        result["temp"] = colls[1].text
        result["press"] = colls[2].text
        result["cloud"] = 0
        for i in cloud_t:
            if colls[3].prettify().find(i) != -1:
                result["cloud"] = cloud_t.index(i)
        result["phen"] = 0
        for i in phen_t:
            if colls[4].prettify().find(i) != -1:
                result["phen"] = phen_t.index(i)
        result["wind_d"] = 0
        for i in wind_d:
            if colls[5].prettify().find(i) != -1:
                result["wind_d"] = wind_d.index(i)
        result["wind"] = ''.join(x for x in colls[5].text if x.isdigit())
        break

In [227]:
result

{'temp': '+10',
 'press': '760',
 'cloud': 4,
 'phen': 0,
 'wind_d': 3,
 'wind': '5'}